In [3]:
!pip install transformers datasets peft accelerate bitsandbytes

  Using cached datasets-3.5.0-py3-none-any.whl.metadata (19 kB)
  Using cached bitsandbytes-0.45.5-py3-none-manylinux_2_24_x86_64.whl.metadata (5.0 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.5.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py311-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manyli

In [13]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write

In [15]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Load exactly 100 samples
dataset = load_dataset("roneneldan/TinyStories", split="train[:100]")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3", use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

# Tokenize data
def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)

tokenized = dataset.map(tokenize, batched=True, remove_columns=["text"])


tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [17]:
from transformers import AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, TaskType

# Load base model in 4-bit mode
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3",
    load_in_4bit=True,
    device_map="auto"
)

# LoRA config
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

trainable params: 3,407,872 || all params: 7,251,431,424 || trainable%: 0.0470


In [18]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./mistral-100stories",
    per_device_train_batch_size=1,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="epoch",
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()


<ipython-input-18-e09ad2b8b39a>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Step,Training Loss
10,1.401300
20,1.489300
30,1.385900
40,1.464300
50,1.454000
60,1.156600
70,1.368400
80,1.187000
90,1.289100
100,1.255100


TrainOutput(global_step=300, training_loss=1.1887219746907551, metrics={'train_runtime': 1297.5445, 'train_samples_per_second': 0.231, 'train_steps_per_second': 0.231, 'total_flos': 6559224142233600.0, 'train_loss': 1.1887219746907551, 'epoch': 3.0})

In [19]:
from transformers import pipeline

# Set up generation pipeline
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Define a simple hallucination checker
def detect_hallucination(story: str) -> str:
    # Rule 1: Does it include age-inappropriate or off-theme content?
    bad_keywords = ["blood", "die", "kill", "weapon", "zombie", "curse", "explode"]
    flagged_words = [word for word in bad_keywords if word in story.lower()]

    # Rule 2: Length sanity
    if len(story.split()) < 30:
        return "Too short – possible failure or hallucination"

    if flagged_words:
        return f"🚨 Possible hallucination! Suspicious words: {', '.join(flagged_words)}"

    return "✅ Looks OK (basic check)"

# Generate and evaluate
prompt = "Once upon a time in a cozy forest"
results = generator(prompt, max_new_tokens=100, temperature=0.8, do_sample=True)

story = results[0]["generated_text"]
print("📖 Story:\n", story)
print("\n🕵️ Hallucination Check:\n", detect_hallucination(story))


Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeo

📖 Story:
 Once upon a time in a cozy forest, there was a little tree. The little tree had long branches that were soft and fluffy. All the animals in the forest liked to play near the little tree.

One day, a clever mouse saw the little tree and wanted to be friends. He climbed up the soft branches and played with his friends. The little tree was happy to have so many friends playing near it. 

The clever mouse wanted to show his friends how clever he was. He showed them a special trick

🕵️ Hallucination Check:
 ✅ Looks OK (basic check)


In [20]:
critique_prompt = f"""
You are an assistant checking stories for babies. Here's a story:
---
{story}
---
Does this story contain any hallucinated, strange, or inappropriate elements for children under 5? Be honest but concise.
"""

critique = generator(critique_prompt, max_new_tokens=100)[0]["generated_text"]
print("\n🤖 Self-Critique:\n", critique)



🤖 Self-Critique:
 
You are an assistant checking stories for babies. Here's a story:
---
Once upon a time in a cozy forest, there was a little tree. The little tree had long branches that were soft and fluffy. All the animals in the forest liked to play near the little tree.

One day, a clever mouse saw the little tree and wanted to be friends. He climbed up the soft branches and played with his friends. The little tree was happy to have so many friends playing near it. 

The clever mouse wanted to show his friends how clever he was. He showed them a special trick
---
Does this story contain any hallucinated, strange, or inappropriate elements for children under 5? Be honest but concise.

No, this story does not contain any hallucinated, strange, or inappropriate elements for children under 5. It is a simple and friendly story about a tree and a mouse playing together.


In [21]:
model.save_pretrained("mistral-100stories-ft")
tokenizer.save_pretrained("mistral-100stories-ft")

('mistral-100stories-ft/tokenizer_config.json',
 'mistral-100stories-ft/special_tokens_map.json',
 'mistral-100stories-ft/tokenizer.model',
 'mistral-100stories-ft/added_tokens.json',
 'mistral-100stories-ft/tokenizer.json')